In [ ]:
# notebooks/06_sentiment_analysis_research.ipynb
# Run in: VS Code or Colab

{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Sentiment Analysis Research for Stock Prediction\n",
    "Analyze news and social media sentiment to enhance trading decisions"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import sys\n",
    "from pathlib import Path\n",
    "import pandas as pd\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "from datetime import datetime, timedelta\n",
    "import warnings\n",
    "warnings.filterwarnings('ignore')\n",
    "\n",
    "project_root = Path.cwd().parent\n",
    "sys.path.insert(0, str(project_root))\n",
    "\n",
    "from src.data.data_loader import DataLoader\n",
    "from src.sentiment.news_scraper import NewsScraper\n",
    "from src.sentiment.sentiment_analyzer import SentimentAnalyzer\n",
    "from app.config import ENABLE_SENTIMENT_ANALYSIS\n",
    "\n",
    "plt.style.use('seaborn-v0_8-darkgrid')\n",
    "%matplotlib inline"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Load Stock Data"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "data_loader = DataLoader()\n",
    "\n",
    "symbols = ['AAPL', 'TSLA', 'GOOGL']\n",
    "stock_data = {}\n",
    "\n",
    "for symbol in symbols:\n",
    "    df = data_loader.load_stock_data(symbol, period='6mo')\n",
    "    if df is not None:\n",
    "        stock_data[symbol] = df\n",
    "        print(f\"{symbol}: Loaded {len(df)} days of data\")\n",
    "\n",
    "symbol = 'AAPL'\n",
    "df = stock_data[symbol]\n",
    "\n",
    "plt.figure(figsize=(14, 6))\n",
    "plt.plot(df.index, df['Close'], linewidth=2)\n",
    "plt.title(f'{symbol} Stock Price (Last 6 Months)', fontweight='bold', fontsize=14)\n",
    "plt.xlabel('Date')\n",
    "plt.ylabel('Price ($)')\n",
    "plt.grid(True, alpha=0.3)\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Sentiment Analysis Setup"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "if not ENABLE_SENTIMENT_ANALYSIS:\n",
    "    print(\"\\n⚠️ WARNING: Sentiment analysis is disabled.\")\n",
    "    print(\"To enable, set NEWSAPI_KEY in your .env file\")\n",
    "    print(\"\\nUsing simulated sentiment data for demonstration...\\n\")\n",
    "    \n",
    "    np.random.seed(42)\n",
    "    dates = pd.date_range(end=datetime.now(), periods=30, freq='D')\n",
    "    \n",
    "    sentiment_data = []\n",
    "    for date in dates:\n",
    "        for symbol in symbols:\n",
    "            sentiment_data.append({\n",
    "                'date': date,\n",
    "                'symbol': symbol,\n",
    "                'sentiment_score': np.random.uniform(-1, 1),\n",
    "                'num_articles': np.random.randint(5, 20),\n",
    "                'source': 'simulated'\n",
    "            })\n",
    "    \n",
    "    sentiment_df = pd.DataFrame(sentiment_data)\n",
    "    print(\"Simulated sentiment data created\")\n",
    "    \n",
    "else:\n",
    "    news_scraper = NewsScraper()\n",
    "    sentiment_analyzer = SentimentAnalyzer()\n",
    "    \n",
    "    sentiment_data = []\n",
    "    \n",
    "    for symbol in symbols:\n",
    "        print(f\"\\nFetching news for {symbol}...\")\n",
    "        articles = news_scraper.fetch_news(symbol, days=30)\n",
    "        \n",
    "        if articles:\n",
    "            print(f\"Found {len(articles)} articles\")\n",
    "            \n",
    "            for article in articles:\n",
    "                sentiment_score = sentiment_analyzer.analyze_text(article['title'] + ' ' + article.get('description', ''))\n",
    "                \n",
    "                sentiment_data.append({\n",
    "                    'date': article['published_at'],\n",
    "                    'symbol': symbol,\n",
    "                    'title': article['title'],\n",
    "                    'sentiment_score': sentiment_score,\n",
    "                    'source': article['source']\n",
    "                })\n",
    "        else:\n",
    "            print(f\"No articles found for {symbol}\")\n",
    "    \n",
    "    sentiment_df = pd.DataFrame(sentiment_data)\n",
    "\n",
    "print(f\"\\nSentiment data shape: {sentiment_df.shape}\")\n",
    "print(sentiment_df.head())"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Sentiment Distribution Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "fig, axes = plt.subplots(1, 3, figsize=(16, 5))\n",
    "\n",
    "for idx, symbol in enumerate(symbols):\n",
    "    symbol_sentiment = sentiment_df[sentiment_df['symbol'] == symbol]\n",
    "    \n",
    "    axes[idx].hist(symbol_sentiment['sentiment_score'], bins=30, edgecolor='black', alpha=0.7)\n",
    "    axes[idx].axvline(x=0, color='r', linestyle='--', linewidth=2, label='Neutral')\n",
    "    axes[idx].axvline(x=symbol_sentiment['sentiment_score'].mean(), color='g', linestyle='--', linewidth=2, label='Mean')\n",
    "    axes[idx].set_title(f'{symbol} Sentiment Distribution', fontweight='bold')\n",
    "    axes[idx].set_xlabel('Sentiment Score')\n",
    "    axes[idx].set_ylabel('Frequency')\n",
    "    axes[idx].legend()\n",
    "    axes[idx].grid(True, alpha=0.3)\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "print(\"\\nSentiment Statistics:\")\n",
    "for symbol in symbols:\n",
    "    symbol_sentiment = sentiment_df[sentiment_df['symbol'] == symbol]['sentiment_score']\n",
    "    print(f\"\\n{symbol}:\")\n",
    "    print(f\"  Mean: {symbol_sentiment.mean():.3f}\")\n",
    "    print(f\"  Std: {symbol_sentiment.std():.3f}\")\n",
    "    print(f\"  Positive: {(symbol_sentiment > 0.1).sum()}\")\n",
    "    print(f\"  Neutral: {((symbol_sentiment >= -0.1) & (symbol_sentiment <= 0.1)).sum()}\")\n",
    "    print(f\"  Negative: {(symbol_sentiment < -0.1).sum()}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Sentiment Time Series"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "sentiment_df['date'] = pd.to_datetime(sentiment_df['date'])\n",
    "\n",
    "daily_sentiment = sentiment_df.groupby(['date', 'symbol'])['sentiment_score'].mean().reset_index()\n",
    "\n",
    "plt.figure(figsize=(14, 6))\n",
    "\n",
    "for symbol in symbols:\n",
    "    symbol_data = daily_sentiment[daily_sentiment['symbol'] == symbol]\n",
    "    plt.plot(symbol_data['date'], symbol_data['sentiment_score'], label=symbol, marker='o', linewidth=2)\n",
    "\n",
    "plt.axhline(y=0, color='gray', linestyle='--', linewidth=1, alpha=0.5)\n",
    "plt.axhline(y=0.3, color='green', linestyle=':', linewidth=1, alpha=0.5, label='Positive Threshold')\n",
    "plt.axhline(y=-0.3, color='red', linestyle=':', linewidth=1, alpha=0.5, label='Negative Threshold')\n",
    "\n",
    "plt.title('Daily Sentiment Scores Over Time', fontweight='bold', fontsize=14)\n",
    "plt.xlabel('Date')\n",
    "plt.ylabel('Average Sentiment Score')\n",
    "plt.legend()\n",
    "plt.grid(True, alpha=0.3)\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Sentiment vs Price Correlation"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "fig, axes = plt.subplots(len(symbols), 1, figsize=(14, 12))\n",
    "\n",
    "for idx, symbol in enumerate(symbols):\n",
    "    df_stock = stock_data[symbol].copy()\n",
    "    df_stock['date'] = df_stock.index\n",
    "    \n",
    "    symbol_sentiment = daily_sentiment[daily_sentiment['symbol'] == symbol].copy()\n",
    "    \n",
    "    merged_df = pd.merge(df_stock, symbol_sentiment, on='date', how='inner')\n",
    "    \n",
    "    ax1 = axes[idx]\n",
    "    ax2 = ax1.twinx()\n",
    "    \n",
    "    ax1.plot(merged_df['date'], merged_df['Close'], color='blue', linewidth=2, label='Stock Price')\n",
    "    ax2.plot(merged_df['date'], merged_df['sentiment_score'], color='orange', linewidth=2, label='Sentiment', alpha=0.7)\n",
    "    ax2.axhline(y=0, color='gray', linestyle='--', linewidth=1, alpha=0.5)\n",
    "    \n",
    "    ax1.set_xlabel('Date')\n",
    "    ax1.set_ylabel('Stock Price ($)', color='blue')\n",
    "    ax2.set_ylabel('Sentiment Score', color='orange')\n",
    "    ax1.set_title(f'{symbol}: Price vs Sentiment', fontweight='bold')\n",
    "    ax1.tick_params(axis='y', labelcolor='blue')\n",
    "    ax2.tick_params(axis='y', labelcolor='orange')\n",
    "    ax1.grid(True, alpha=0.3)\n",
    "    \n",
    "    lines1, labels1 = ax1.get_legend_handles_labels()\n",
    "    lines2, labels2 = ax2.get_legend_handles_labels()\n",
    "    ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')\n",
    "    \n",
    "    if len(merged_df) > 1:\n",
    "        correlation = merged_df['Close'].corr(merged_df['sentiment_score'])\n",
    "        ax1.text(0.02, 0.98, f'Correlation: {correlation:.3f}', \n",
    "                transform=ax1.transAxes, verticalalignment='top',\n",
    "                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6. Sentiment-Based Trading Signals"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def generate_sentiment_signals(df, sentiment_df, symbol, threshold=0.3):\n",
    "    df = df.copy()\n",
    "    df['date'] = df.index\n",
    "    \n",
    "    symbol_sentiment = daily_sentiment[daily_sentiment['symbol'] == symbol].copy()\n",
    "    merged = pd.merge(df, symbol_sentiment, on='date', how='left')\n",
    "    merged['sentiment_score'].fillna(0, inplace=True)\n",
    "    \n",
    "    merged['signal'] = 'HOLD'\n",
    "    merged.loc[merged['sentiment_score'] > threshold, 'signal'] = 'BUY'\n",
    "    merged.loc[merged['sentiment_score'] < -threshold, 'signal'] = 'SELL'\n",
    "    \n",
    "    return merged\n",
    "\n",
    "symbol = 'AAPL'\n",
    "signals_df = generate_sentiment_signals(stock_data[symbol], daily_sentiment, symbol)\n",
    "\n",
    "plt.figure(figsize=(14, 8))\n",
    "\n",
    "plt.subplot(2, 1, 1)\n",
    "plt.plot(signals_df['date'], signals_df['Close'], linewidth=2, label='Price')\n",
    "\n",
    "buy_signals = signals_df[signals_df['signal'] == 'BUY']\n",
    "sell_signals = signals_df[signals_df['signal'] == 'SELL']\n",
    "\n",
    "plt.scatter(buy_signals['date'], buy_signals['Close'], color='green', marker='^', s=100, label='BUY Signal', zorder=5)\n",
    "plt.scatter(sell_signals['date'], sell_signals['Close'], color='red', marker='v', s=100, label='SELL Signal', zorder=5)\n",
    "\n",
    "plt.title(f'{symbol} Sentiment-Based Trading Signals', fontweight='bold')\n",
    "plt.ylabel('Price ($)')\n",
    "plt.legend()\n",
    "plt.grid(True, alpha=0.3)\n",
    "\n",
    "plt.subplot(2, 1, 2)\n",
    "plt.plot(signals_df['date'], signals_df['sentiment_score'], linewidth=2, color='purple')\n",
    "plt.axhline(y=0.3, color='green', linestyle='--', label='Buy Threshold')\n",
    "plt.axhline(y=-0.3, color='red', linestyle='--', label='Sell Threshold')\n",
    "plt.axhline(y=0, color='gray', linestyle='-', alpha=0.3)\n",
    "\n",
    "plt.title('Sentiment Score', fontweight='bold')\n",
    "plt.xlabel('Date')\n",
    "plt.ylabel('Sentiment')\n",
    "plt.legend()\n",
    "plt.grid(True, alpha=0.3)\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "print(f\"\\nSignal Summary for {symbol}:\")\n",
    "print(f\"BUY signals: {len(buy_signals)}\")\n",
    "print(f\"SELL signals: {len(sell_signals)}\")\n",
    "print(f\"HOLD periods: {(signals_df['signal'] == 'HOLD').sum()}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 7. Sentiment-Enhanced Backtest"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def backtest_sentiment_strategy(signals_df, initial_capital=10000):\n",
    "    capital = initial_capital\n",
    "    position = 0\n",
    "    trades = []\n",
    "    portfolio_values = []\n",
    "    \n",
    "    for idx, row in signals_df.iterrows():\n",
    "        if row['signal'] == 'BUY' and position == 0:\n",
    "            shares = capital / row['Close']\n",
    "            position = shares\n",
    "            capital = 0\n",
    "            trades.append({'date': row['date'], 'type': 'BUY', 'price': row['Close'], 'shares': shares})\n",
    "        \n",
    "        elif row['signal'] == 'SELL' and position > 0:\n",
    "            capital = position * row['Close']\n",
    "            trades.append({'date': row['date'], 'type': 'SELL', 'price': row['Close'], 'shares': position})\n",
    "            position = 0\n",
    "        \n",
    "        portfolio_value = capital + (position * row['Close'])\n",
    "        portfolio_values.append({'date': row['date'], 'value': portfolio_value})\n",
    "    \n",
    "    if position > 0:\n",
    "        capital = position * signals_df.iloc[-1]['Close']\n",
    "    \n",
    "    return {\n",
    "        'final_value': capital,\n",
    "        'trades': trades,\n",
    "        'portfolio_values': portfolio_values\n",
    "    }\n",
    "\n",
    "backtest_results = backtest_sentiment_strategy(signals_df)\n",
    "\n",
    "initial_capital = 10000\n",
    "final_value = backtest_results['final_value']\n",
    "total_return = ((final_value - initial_capital) / initial_capital) * 100\n",
    "\n",
    "buy_hold_value = (initial_capital / signals_df.iloc[0]['Close']) * signals_df.iloc[-1]['Close']\n",
    "buy_hold_return = ((buy_hold_value - initial_capital) / initial_capital) * 100\n",
    "\n",
    "print(f\"\\n{symbol} Sentiment Strategy Backtest Results:\")\n",
    "print(f\"Initial Capital: ${initial_capital:,.2f}\")\n",
    "print(f\"Final Value: ${final_value:,.2f}\")\n",
    "print(f\"Total Return: {total_return:.2f}%\")\n",
    "print(f\"Number of Trades: {len(backtest_results['trades'])}\")\n",
    "print(f\"\\nBuy & Hold Return: {buy_hold_return:.2f}%\")\n",
    "print(f\"Sentiment Strategy Outperformance: {total_return - buy_hold_return:.2f}%\")\n",
    "\n",
    "portfolio_df = pd.DataFrame(backtest_results['portfolio_values'])\n",
    "\n",
    "plt.figure(figsize=(14, 6))\n",
    "plt.plot(portfolio_df['date'], portfolio_df['value'], linewidth=2, label='Sentiment Strategy')\n",
    "\n",
    "buy_hold_values = [(initial_capital / signals_df.iloc[0]['Close']) * price \n",
    "                   for price in signals_df['Close']]\n",
    "plt.plot(signals_df['date'], buy_hold_values, linewidth=2, alpha=0.7, label='Buy & Hold')\n",
    "\n",
    "plt.axhline(y=initial_capital, color='r', linestyle='--', alpha=0.5, label='Initial Capital')\n",
    "\n",
    "plt.title('Sentiment Strategy vs Buy & Hold', fontweight='bold', fontsize=14)\n",
    "plt.xlabel('Date')\n",
    "plt.ylabel('Portfolio Value ($)')\n",
    "plt.legend()\n",
    "plt.grid(True, alpha=0.3)\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 8. Multi-Stock Sentiment Comparison"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "sentiment_summary = []\n",
    "\n",
    "for symbol in symbols:\n",
    "    symbol_sentiment = sentiment_df[sentiment_df['symbol'] == symbol]['sentiment_score']\n",
    "    \n",
    "    sentiment_summary.append({\n",
    "        'Symbol': symbol,\n",
    "        'Avg Sentiment': symbol_sentiment.mean(),\n",
    "        'Std Dev': symbol_sentiment.std(),\n",
    "        'Min': symbol_sentiment.min(),\n",
    "        'Max': symbol_sentiment.max(),\n",
    "        'Positive %': (symbol_sentiment > 0.1).sum() / len(symbol_sentiment) * 100,\n",
    "        'Negative %': (symbol_sentiment < -0.1).sum() / len(symbol_sentiment) * 100\n",
    "    })\n",
    "\n",
    "sentiment_summary_df = pd.DataFrame(sentiment_summary)\n",
    "\n",
    "print(\"\\nSentiment Summary Across Stocks:\")\n",
    "print(sentiment_summary_df.to_string(index=False))\n",
    "\n",
    "fig, axes = plt.subplots(1, 2, figsize=(14, 6))\n",
    "\n",
    "sentiment_summary_df.plot(x='Symbol', y='Avg Sentiment', kind='bar', ax=axes[0], legend=False, color='steelblue')\n",
    "axes[0].axhline(y=0, color='r', linestyle='--', linewidth=1)\n",
    "axes[0].set_title('Average Sentiment by Stock', fontweight='bold')\n",
    "axes[0].set_ylabel('Sentiment Score')\n",
    "axes[0].set_xlabel('Stock Symbol')\n",
    "axes[0].grid(True, alpha=0.3)\n",
    "\n",
    "sentiment_summary_df[['Symbol', 'Positive %', 'Negative %']].set_index('Symbol').plot(kind='bar', ax=axes[1], stacked=False)\n",
    "axes[1].set_title('Sentiment Distribution by Stock', fontweight='bold')\n",
    "axes[1].set_ylabel('Percentage (%)')\n",
    "axes[1].set_xlabel('Stock Symbol')\n",
    "axes[1].legend(['Positive', 'Negative'])\n",
    "axes[1].grid(True, alpha=0.3)\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 9. Sentiment Lead/Lag Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def calculate_sentiment_price_lag(df, sentiment_df, symbol, max_lag=5):\n",
    "    df = df.copy()\n",
    "    df['date'] = df.index\n",
    "    df['returns'] = df['Close'].pct_change()\n",
    "    \n",
    "    symbol_sentiment = daily_sentiment[daily_sentiment['symbol'] == symbol].copy()\n",
    "    merged = pd.merge(df, symbol_sentiment, on='date', how='inner')\n",
    "    \n",
    "    correlations = []\n",
    "    \n",
    "    for lag in range(-max_lag, max_lag + 1):\n",
    "        if lag < 0:\n",
    "            shifted_returns = merged['returns'].shift(-lag)\n",
    "            corr = merged['sentiment_score'].corr(shifted_returns)\n",
    "        else:\n",
    "            shifted_sentiment = merged['sentiment_score'].shift(lag)\n",
    "            corr = shifted_sentiment.corr(merged['returns'])\n",
    "        \n",
    "        correlations.append({'lag': lag, 'correlation': corr})\n",
    "    \n",
    "    return pd.DataFrame(correlations)\n",
    "\n",
    "fig, axes = plt.subplots(len(symbols), 1, figsize=(12, 10))\n",
    "\n",
    "for idx, symbol in enumerate(symbols):\n",
    "    lag_df = calculate_sentiment_price_lag(stock_data[symbol], sentiment_df, symbol)\n",
    "    \n",
    "    axes[idx].bar(lag_df['lag'], lag_df['correlation'], alpha=0.7)\n",
    "    axes[idx].axhline(y=0, color='r', linestyle='--', linewidth=1)\n",
    "    axes[idx].axvline(x=0, color='gray', linestyle='--', linewidth=1, alpha=0.5)\n",
    "    axes[idx].set_title(f'{symbol}: Sentiment-Price
    Correlation (Lag Analysis)', fontweight='bold')\n",
"    axes[idx].set_xlabel('Lag (days, negative = sentiment leads)')\n",
"    axes[idx].set_ylabel('Correlation')\n",
"    axes[idx].grid(True, alpha=0.3)\n",
"    \n",
"    max_corr_idx = lag_df['correlation'].abs().idxmax()\n",
"    max_lag = lag_df.loc[max_corr_idx, 'lag']\n",
"    max_corr = lag_df.loc[max_corr_idx, 'correlation']\n",
"    \n",
"    if max_lag < 0:\n",
"        lead_text = f"Sentiment leads by {abs(max_lag)} days"\n",
"    elif max_lag > 0:\n",
"        lead_text = f"Price leads by {max_lag} days"\n",
"    else:\n",
"        lead_text = "Simultaneous"\n",
"    \n",
"    axes[idx].text(0.02, 0.98, f'{lead_text}\nCorr: {max_corr:.3f}',\n",
"                  transform=axes[idx].transAxes, verticalalignment='top',\n",
"                  bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))\n",
"\n",
"plt.tight_layout()\n",
"plt.show()"
]
},
{
"cell_type": "markdown",
"metadata": {},
"source": [
"## 10. Conclusions and Recommendations"
]
},
{
"cell_type": "code",
"execution_count": null,
"metadata": {},
"outputs": [],
"source": [
"print("\n" + "="*80)\n",
"print("SENTIMENT ANALYSIS RESEARCH SUMMARY")\n",
"print("="*80)\n",
"\n",
"print("\n1. Key Findings:")\n",
"print("   - Sentiment scores show varying correlation with stock prices")\n",
"print("   - Extreme sentiment (very positive/negative) can signal trading opportunities")\n",
"print("   - Sentiment may lead or lag price movements depending on the stock")\n",
"print("   - Combining sentiment with technical analysis improves signal quality")\n",
"\n",
"print("\n2. Performance Insights:")\n",
"print(f"   - Sentiment strategy return: {total_return:.2f}%")\n",
"print(f"   - Buy & Hold return: {buy_hold_return:.2f}%")\n",
"print(f"   - Outperformance: {total_return - buy_hold_return:.2f}%")\n",
"\n",
"print("\n3. Recommendations:")\n",
"print("   - Use sentiment as a complementary signal, not standalone")\n",
"print("   - Monitor sentiment from multiple sources for better accuracy")\n",
"print("   - Adjust sentiment thresholds based on market volatility")\n",
"print("   - Consider sentiment momentum (rate of change) for timing")\n",
"print("   - Validate sentiment signals with fundamental and technical analysis")\n",
"\n",
"print("\n4. Limitations:")\n",
"print("   - Sentiment data quality depends on news source availability")\n",
"print("   - Delayed news may reduce prediction accuracy")\n",
"print("   - Market manipulation through fake news is a risk")\n",
"print("   - Past sentiment-price relationships may not persist")\n",
"\n",
"print("\n5. Future Work:")\n",
"print("   - Incorporate social media sentiment (Twitter, Reddit)")\n",
"print("   - Use transformer models (BERT, GPT) for better sentiment extraction")\n",
"print("   - Analyze sentiment by news category (earnings, products, legal)")\n",
"print("   - Build real-time sentiment monitoring dashboard")\n",
"\n",
"print("\n" + "="*80)"
]
}
],
"metadata": {
"kernelspec": {
"display_name": "Python 3",
"language": "python",
"name": "python3"
},
"language_info": {
"codemirror_mode": {
"name": "ipython",
"version": 3
},
"file_extension": ".py",
"mimetype": "text/x-python",
"name": "python",
"nbconvert_exporter": "python",
"pygments_lexer": "ipython3",
"version": "3.9.0"
}
},
"nbformat": 4,
"nbformat_minor": 4
}